In [12]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
import math

# Parameters
learning_rate = 0.0001 #0.001
training_epochs = 120 #15
batch_size = 100
display_step = 1

In [2]:
import csv
import os
import numpy as np

#인코딩중요
#라벨만들기
with open('./trainSet.csv', 'r', encoding='UTF-8-sig') as raw:
    lines = raw.readlines()
data = list(csv.reader(lines))
data = sorted(data, key=lambda x: x[0])

#이름빼기
_Y = [y[1] for y in data]

_Y = np.array(_Y)
_Y = _Y.astype(float) # shape(-1)
_Y = [[1-y, y] for y in _Y] # shape(-1, 2)



#데이터만들기
with open('./features4096.csv', 'r', encoding='UTF-8-sig') as raw:
    lines = raw.readlines()
data = list(csv.reader(lines))
data = sorted(data, key=lambda x: x[0])

#이름빼기
_X = [x[1:] for x in data]
_X = np.array(_X)
_X = _X.astype(float) # shape(-1, 1024)



#데이터만들기
with open('./features_3g_16.csv', 'r', encoding='UTF-8-sig') as raw:
    lines = raw.readlines()
data = list(csv.reader(lines))
data = sorted(data, key=lambda x: x[0])

#이름빼기
_Z = [x[1:] for x in data]
_Z = np.array(_Z)
_Z = _Z.astype(float) # shape(-1, 1024)



#데이터만들기
with open('./features_4g_8.csv', 'r', encoding='UTF-8-sig') as raw:
    lines = raw.readlines()
data = list(csv.reader(lines))
data = sorted(data, key=lambda x: x[0])

#이름빼기
_W = [x[1:] for x in data]
_W = np.array(_W)
_W = _W.astype(float) # shape(-1, 1024)



all_set = tuple(zip(_X, _Z, _W, _Y)) # shape(10000, 4, ~)
M_all_set = [all_set[i] for i in range(len(all_set)) if all_set[i][-1][1] == 1] # 7000
N_all_set = [all_set[i] for i in range(len(all_set)) if all_set[i][-1][0] == 1] # 3000
np.random.shuffle(M_all_set)
np.random.shuffle(N_all_set)

# 악성,정상 개수 맞추기
if len(M_all_set) > len(N_all_set):
    M_all_set = M_all_set[:len(N_all_set)]
else:
    N_all_set = N_all_set[:len(M_all_set)]

all_set = M_all_set + N_all_set
np.random.shuffle(all_set) # 6000

train_set = all_set[len(all_set)//5:] # 4800
test_set = all_set[:len(all_set)//5] # 1200

train_X = np.array([X for X,Z,W,Y in train_set])
train_Z = np.array([Z for X,Z,W,Y in train_set])
train_W = np.array([W for X,Z,W,Y in train_set])
train_Y = np.array([Y for X,Z,W,Y in train_set])
test_X = np.array([X for X,Z,W,Y in test_set])
test_Z = np.array([Z for X,Z,W,Y in test_set])
test_W = np.array([W for X,Z,W,Y in test_set])
test_Y = np.array([Y for X,Z,W,Y in test_set])


M_test_X = [X for X,Z,W,Y in test_set if Y[1] == 1] # label의 악성이면
M_test_Z = [Z for X,Z,W,Y in test_set if Y[1] == 1] # label의 악성이면
M_test_W = [W for X,Z,W,Y in test_set if Y[1] == 1] # label의 악성이면
M_test_Y = [Y for X,Z,W,Y in test_set if Y[1] == 1] # label의 악성이면

N_test_X = [X for X,Z,W,Y in test_set if Y[0] == 1] # label의 정상이면
N_test_Z = [Z for X,Z,W,Y in test_set if Y[0] == 1] # label의 정상이면
N_test_W = [W for X,Z,W,Y in test_set if Y[0] == 1] # label의 정상이면
N_test_Y = [Y for X,Z,W,Y in test_set if Y[0] == 1] # label의 정상이면




In [3]:
with tf.device('/gpu:0'):
    X = tf.placeholder(tf.float32, [None, 4096])
    Z = tf.placeholder(tf.float32, [None, 4096])
    W = tf.placeholder(tf.float32, [None, 4096])
    Y = tf.placeholder(tf.float32, [None, 2])
    keep_prob = tf.placeholder(tf.float32)

    #X = tf.nn.dropout(X, keep_prob=keep_prob)

    # Layer1
    W1_1 = tf.get_variable('%f'%(np.random.random()), shape=[4096,2000], initializer=tf.contrib.layers.xavier_initializer())
    b1_1 = tf.Variable(tf.random_normal([2000]))
    L1_1 = tf.nn.relu(tf.matmul(X, W1_1) + b1_1)
    L1_1 = tf.nn.dropout(L1_1, keep_prob=keep_prob)

    # Layer2
    W2_1 = tf.get_variable('%f'%(np.random.random()), shape=[2000,1000], initializer=tf.contrib.layers.xavier_initializer())
    b2_1 = tf.Variable(tf.random_normal([1000]))
    L2_1 = tf.nn.relu(tf.matmul(L1_1, W2_1) + b2_1)
    L2_1 = tf.nn.dropout(L2_1, keep_prob=keep_prob)

    # Layer3
    W3_1 = tf.get_variable('%f'%(np.random.random()), shape=[1000,500], initializer=tf.contrib.layers.xavier_initializer())
    b3_1 = tf.Variable(tf.random_normal([500]))
    L3_1 = tf.nn.relu(tf.matmul(L2_1, W3_1) + b3_1)
    L3_1 = tf.nn.dropout(L3_1, keep_prob=keep_prob)

    # Layer4
    W4_1 = tf.get_variable('%f'%(np.random.random()), shape=[500,250], initializer=tf.contrib.layers.xavier_initializer())
    b4_1 = tf.Variable(tf.random_normal([250]))
    L4_1 = tf.nn.relu(tf.matmul(L3_1, W4_1) + b4_1)
    L4_1 = tf.nn.dropout(L4_1, keep_prob=keep_prob)
    
    # Layer5
    W5_1 = tf.get_variable('%f'%(np.random.random()), shape=[250,100], initializer=tf.contrib.layers.xavier_initializer())
    b5_1 = tf.Variable(tf.random_normal([100]))
    L5_1 = tf.nn.relu(tf.matmul(L4_1, W5_1) + b5_1)
    L5_1 = tf.nn.dropout(L5_1, keep_prob=keep_prob)
    
    # Layer6
    W6_1 = tf.get_variable('%f'%(np.random.random()), shape=[100,2], initializer=tf.contrib.layers.xavier_initializer())
    b6_1 = tf.Variable(tf.random_normal([2]))
    hypothesis_1 = tf.matmul(L5_1, W6_1) + b6_1
    
    
    # Layer1
    W1_2 = tf.get_variable('%f'%(np.random.random()), shape=[4096,2000], initializer=tf.contrib.layers.xavier_initializer())
    b1_2 = tf.Variable(tf.random_normal([2000]))
    L1_2 = tf.nn.relu(tf.matmul(Z, W1_2) + b1_2)
    L1_2 = tf.nn.dropout(L1_2, keep_prob=keep_prob)

    # Layer2
    W2_2 = tf.get_variable('%f'%(np.random.random()), shape=[2000,1000], initializer=tf.contrib.layers.xavier_initializer())
    b2_2 = tf.Variable(tf.random_normal([1000]))
    L2_2 = tf.nn.relu(tf.matmul(L1_2, W2_2) + b2_2)
    L2_2 = tf.nn.dropout(L2_2, keep_prob=keep_prob)

    # Layer3
    W3_2 = tf.get_variable('%f'%(np.random.random()), shape=[1000,500], initializer=tf.contrib.layers.xavier_initializer())
    b3_2 = tf.Variable(tf.random_normal([500]))
    L3_2 = tf.nn.relu(tf.matmul(L2_2, W3_2) + b3_2)
    L3_2 = tf.nn.dropout(L3_2, keep_prob=keep_prob)

    # Layer4
    W4_2 = tf.get_variable('%f'%(np.random.random()), shape=[500,250], initializer=tf.contrib.layers.xavier_initializer())
    b4_2 = tf.Variable(tf.random_normal([250]))
    L4_2 = tf.nn.relu(tf.matmul(L3_2, W4_2) + b4_2)
    L4_2 = tf.nn.dropout(L4_2, keep_prob=keep_prob)
    
    # Layer5
    W5_2 = tf.get_variable('%f'%(np.random.random()), shape=[250,100], initializer=tf.contrib.layers.xavier_initializer())
    b5_2 = tf.Variable(tf.random_normal([100]))
    L5_2 = tf.nn.relu(tf.matmul(L4_2, W5_2) + b5_2)
    L5_2 = tf.nn.dropout(L5_2, keep_prob=keep_prob)
    
    # Layer6
    W6_2 = tf.get_variable('%f'%(np.random.random()), shape=[100,2], initializer=tf.contrib.layers.xavier_initializer())
    b6_2 = tf.Variable(tf.random_normal([2]))
    hypothesis_2 = tf.matmul(L5_2, W6_2) + b6_2
    
    
    # Layer1
    W1_3 = tf.get_variable('%f'%(np.random.random()), shape=[4096,2000], initializer=tf.contrib.layers.xavier_initializer())
    b1_3 = tf.Variable(tf.random_normal([2000]))
    L1_3 = tf.nn.relu(tf.matmul(W, W1_3) + b1_3)
    L1_3 = tf.nn.dropout(L1_3, keep_prob=keep_prob)

    # Layer2
    W2_3 = tf.get_variable('%f'%(np.random.random()), shape=[2000,1000], initializer=tf.contrib.layers.xavier_initializer())
    b2_3 = tf.Variable(tf.random_normal([1000]))
    L2_3 = tf.nn.relu(tf.matmul(L1_3, W2_3) + b2_3)
    L2_3 = tf.nn.dropout(L2_3, keep_prob=keep_prob)

    # Layer3
    W3_3 = tf.get_variable('%f'%(np.random.random()), shape=[1000,500], initializer=tf.contrib.layers.xavier_initializer())
    b3_3 = tf.Variable(tf.random_normal([500]))
    L3_3 = tf.nn.relu(tf.matmul(L2_3, W3_3) + b3_3)
    L3_3 = tf.nn.dropout(L3_3, keep_prob=keep_prob)

    # Layer4
    W4_3 = tf.get_variable('%f'%(np.random.random()), shape=[500,250], initializer=tf.contrib.layers.xavier_initializer())
    b4_3 = tf.Variable(tf.random_normal([250]))
    L4_3 = tf.nn.relu(tf.matmul(L3_3, W4_3) + b4_3)
    L4_3 = tf.nn.dropout(L4_3, keep_prob=keep_prob)
    
    # Layer5
    W5_3 = tf.get_variable('%f'%(np.random.random()), shape=[250,100], initializer=tf.contrib.layers.xavier_initializer())
    b5_3 = tf.Variable(tf.random_normal([100]))
    L5_3 = tf.nn.relu(tf.matmul(L4_3, W5_3) + b5_3)
    L5_3 = tf.nn.dropout(L5_3, keep_prob=keep_prob)
    
    # Layer6
    W6_3 = tf.get_variable('%f'%(np.random.random()), shape=[100,2], initializer=tf.contrib.layers.xavier_initializer())
    b6_3 = tf.Variable(tf.random_normal([2]))
    hypothesis_3 = tf.matmul(L5_3, W6_3) + b6_3
    
    

    cost1 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis_1, labels=Y))
    optimizer1 = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost1)
    
    cost2 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis_2, labels=Y))
    optimizer2 = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost2)
    
    cost3 = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis_3, labels=Y))
    optimizer3 = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost3)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [19]:
# Training
sess = tf.Session()
sess.run(tf.global_variables_initializer())
#sess.run(tf.initialize_all_variables())

# Training cycle
for epoch in range(training_epochs):
    avg_cost1 = 0
    avg_cost2 = 0
    avg_cost3 = 0
    total_batch = int(len(train_X) / batch_size)

    for i in range(total_batch):
        batch_xs, batch_zs, batch_ws, batch_ys = train_X[i*batch_size:(i+1)*batch_size], \
                                        train_Z[i*batch_size:(i+1)*batch_size], \
                                        train_W[i*batch_size:(i+1)*batch_size], \
                                        train_Y[i*batch_size:(i+1)*batch_size]
        feed_dict = {X: batch_xs, Z: batch_zs, W: batch_ws, Y: batch_ys, keep_prob: 0.7}
        #feed_dict = {X: batch_xs, Y: batch_ys}
        c1, _, c2, _, c3, _ = sess.run([cost1, optimizer1, cost2, optimizer2, cost3, optimizer3], feed_dict=feed_dict)
        #c1, _, c2, _ = sess.run([cost1, optimizer1, cost2, optimizer2], feed_dict=feed_dict)
        
        avg_cost1 += c1 / total_batch #뭉치갯수로 나누기
        avg_cost2 += c2 / total_batch #뭉치갯수로 나누기
        avg_cost3 += c3 / total_batch #뭉치갯수로 나누기
        time.sleep(0.01)
        
    if (epoch+1)%5 == 0:
        print('Epoch: %03d'%(epoch+1), 
              '\tcost1=', '{:.9f}'.format(avg_cost1), 
              '\tcost2=', '{:.9f}'.format(avg_cost2),
              '\tcost3=', '{:.9f}'.format(avg_cost3))


Epoch: 005 	cost1= 0.591802514 	cost2= 0.580696532 	cost3= 0.601735126
Epoch: 010 	cost1= 0.407409998 	cost2= 0.403537552 	cost3= 0.425349379
Epoch: 015 	cost1= 0.330225491 	cost2= 0.326812250 	cost3= 0.310444323
Epoch: 020 	cost1= 0.270341549 	cost2= 0.249529573 	cost3= 0.248740984
Epoch: 025 	cost1= 0.226210420 	cost2= 0.218632134 	cost3= 0.208262527
Epoch: 030 	cost1= 0.214466002 	cost2= 0.177888841 	cost3= 0.165192039
Epoch: 035 	cost1= 0.175372000 	cost2= 0.172269834 	cost3= 0.150427185
Epoch: 040 	cost1= 0.151458168 	cost2= 0.158458196 	cost3= 0.165346589
Epoch: 045 	cost1= 0.134582029 	cost2= 0.158322581 	cost3= 0.133040805
Epoch: 050 	cost1= 0.161614551 	cost2= 0.100608695 	cost3= 0.128216042
Epoch: 055 	cost1= 0.110669108 	cost2= 0.096759951 	cost3= 0.136002837
Epoch: 060 	cost1= 0.101823651 	cost2= 0.105627223 	cost3= 0.138247033
Epoch: 065 	cost1= 0.123846051 	cost2= 0.099691486 	cost3= 0.098681871
Epoch: 070 	cost1= 0.106552041 	cost2= 0.087101284 	cost3= 0.090004757
Epoch:

In [20]:
# TEST
#is_correct = tf.equal(tf.arg_max(hypothesis_1 + hypothesis_2 + hypothesis_3, 1), tf.arg_max(Y, 1))
is_correct = tf.equal(tf.arg_max(tf.round(tf.nn.softmax(hypothesis_1)) + \
                                 tf.round(tf.nn.softmax(hypothesis_2)) + \
                                 tf.round(tf.nn.softmax(hypothesis_3)), 1), tf.arg_max(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

#feed_dict = {X: test_X, Y: test_Y, keep_prob: 1}
#print("Accuracy : ", accuracy.eval(session=sess, feed_dict=feed_dict))

#feed_dict = {X: M_test_X, Y: M_test_Y, keep_prob: 1}
#print("정탐율 : ", accuracy.eval(session=sess, feed_dict=feed_dict))

#feed_dict = {X: N_test_X, Y: N_test_Y, keep_prob: 1}
#print("오탐율 : ", 1 - accuracy.eval(session=sess, feed_dict=feed_dict))

total_batch = math.ceil(len(test_Y) / batch_size)
total_acc = 0
for i in range(total_batch):
    batch_xs, batch_zs, batch_ws, batch_ys = test_X[i*batch_size:(i+1)*batch_size], \
                                    test_Z[i*batch_size:(i+1)*batch_size], \
                                    test_W[i*batch_size:(i+1)*batch_size], \
                                    test_Y[i*batch_size:(i+1)*batch_size]
    feed_dict = {X: batch_xs, Z: batch_zs, W: batch_ws, Y: batch_ys, keep_prob: 1}
    one_acc = accuracy.eval(session=sess, feed_dict=feed_dict)
    total_acc += one_acc * len(batch_ys)
print("Accuracy : %0.4f"%(total_acc/len(test_Y)))


total_batch = math.ceil(len(M_test_Y) / batch_size)
total_acc = 0
for i in range(total_batch):
    batch_xs, batch_zs, batch_ws, batch_ys = M_test_X[i*batch_size:(i+1)*batch_size], \
                                    M_test_Z[i*batch_size:(i+1)*batch_size], \
                                    M_test_W[i*batch_size:(i+1)*batch_size], \
                                    M_test_Y[i*batch_size:(i+1)*batch_size]
    feed_dict = {X: batch_xs, Z: batch_zs, W: batch_ws, Y: batch_ys, keep_prob: 1}
    one_acc = accuracy.eval(session=sess, feed_dict=feed_dict)
    total_acc += one_acc * len(batch_ys)
print("정탐율 : %0.4f"%(total_acc/len(M_test_Y)))


total_batch = math.ceil(len(N_test_Y) / batch_size)
total_acc = 0
for i in range(total_batch):
    batch_xs, batch_zs, batch_ws, batch_ys = N_test_X[i*batch_size:(i+1)*batch_size], \
                                    N_test_Z[i*batch_size:(i+1)*batch_size], \
                                    N_test_W[i*batch_size:(i+1)*batch_size], \
                                    N_test_Y[i*batch_size:(i+1)*batch_size]
    feed_dict = {X: batch_xs, Z: batch_zs, W: batch_ws, Y: batch_ys, keep_prob: 1}
    one_acc = accuracy.eval(session=sess, feed_dict=feed_dict)
    total_acc += one_acc * len(batch_ys)
print("오탐율 : %0.4f"%(total_acc/len(N_test_Y)))



#r = rd.randint(0, len(test_X) - 1)
#print('Label:', sess.run(tf.argmax(test_Y[r:r+1], 1)))

#feed_dict = {X: test_X[r:r+1], keep_prob: 1}
#print('Prediction:', sess.run(tf.argmax(hypothesis, 1), feed_dict=feed_dict))

Accuracy : 0.9450
정탐율 : 0.9506
오탐율 : 0.9396
